TOPIC RECOGNITION

ci sono 3 importanti parametri:
-numero di topic
-numero di parole per topic
-numero di topic per documento

STEP:
-TOKENIZZARE E RIPULIRE OGNI DOCUMENTO (QUINDI OGNI POST)(utilizziamo tokenizer per fare prima questa volta)
Lasciare #parola?? (boh decidere)

-per farlo con LSA il prof usa sklearn



In [ ]:
import funzioni_progetto_webdata as fpw

In [ ]:
from google.colab import files
 
 
uploaded = files.upload()

In [ ]:
# import csv
# from collections import Counter
# from tkinter.tix import COLUMN
# import matplotlib.pyplot as plt
# import re
# import pandas as pd
# from math import log2
# import spacy
# import nltk
# nltk.download('omw-1.4')
# nltk.download('wordnet')
# from nltk.stem import WordNetLemmatizer
# import gensim, pprint



In [2]:
corpus=["https://colab.research.google.com/drive/1dduOM7M3RubSeqpogrM1H7PhL2S9ZMbt#scrollTo=1eeVrjeBpn1x",
        "is not happy capitol riot vote forces , not happy Republican to pick truth or Trump not good is not yours",
        "EU not good agrees to allow entry to vaccinated travelers",
        "Why the United Nation is stuck on the Israeli-Palestinian conflict",
        "New York attorney general adds 'criminal capacity' to probe of Trump Organization",
        "Given the recent accusations against the European Commission on land registry, beach resorts, PNRR, public accounts, etc., it is clear that many think that the EU should behave as if it were our ATM that gives money and is silent. But the ATM gives you your money, the EU gives us other people's money.",
        "ANSA FORUM | Lega secretary Matteo Salvini responds live to questions from director Luigi Contu and colleague of the political-parliamentary editorial staff Michela Suglia #ANSA",
        "Murder Willy, Bianchi in the courtroom: I didn't even touch him with a finger",
        "The positives at #vaiolodellescimmie in Lombardy have risen to three. The Region has already communicated the protocol that hospitals must follow in the event of symptoms appearing",
        "An assault rifle in the hand of an 18-year-old. An assault rifle. #Texas",
        "Transfer market live: Pogba-Juve in the final close, Napoli and Fiorentina on Deulofeu [by Giulio Cardone, Enrico De Lellis]",
        "#Economia circular and #gestione environmental: limits and opportunities. The #aziende must carefully screen #investimenti , #costi and perspectives and must show #flessibilità and a rapid propensity for change",
        "Fire on the island of Stromboli, flames extinguished after a night of fire #ANSA",
        "A new anti-cancer vaccine, tested in mice and monkeys, has been shown to be able to break down the defenses that tumors put in place to protect themselves from attacks by the immune system by blocking diseased cells",
        "Term of the day: Computational fluid dynamics Definition: The numerical solution of flow equations in practical problems such as aircraft design or hydraulic structures.",
        "The mayor of Kiev: 'The Russian war will not stop in Ukraine'",
        "The killer of elementary school in #Uvalde in #Texas Salvador Ramos worked in a restaurant not far from the scene of the massacre. He would have been able to save around 4,000 dollars (3.7 thousand euros) to buy the weapons he used in the massacre"]

In [8]:
bigram_mdl,vectorizer,concetti=fpw.create_lsa_model(corpus)

#parole=list(gensim.utils.tokenize('the war is a universal conflict ukraine fire', lower=True))#usare poi la funzione che pulisce bene come per iol topic
lista_token = gensim.utils.tokenize(remove_link_menzione("the war is a universal conflict ukraine fire"), lower=True)
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words
wnl = WordNetLemmatizer()
lemmatized_string=[wnl.lemmatize(words) for words in find_and_merge_not(lista_token,all_stopwords)]
post=bigram_mdl[lemmatized_string]


punteggi_LSA=fpw.punteggi_LSA(concetti,parole)

risultatoTopicLSA=fpw.result_TopicLSA(punteggi_LSA,concetti)
risultatoTopicLSAMax=fpw.result_Topic_max(punteggi_LSA,concetti)

LDA

In [ ]:
ldamodel,concetti_LDA,corpus2,dictionary=fpw.lda(post)

punteggi_LDA=fpw.punteggioLDA(concetti_LDA,parole)

risultatoTopicMax=fpw.result_TopicLDA(concetti_LDA)

#fpw.grafico_LDA(ldamodel,corpus2,dictionary)

HDP

In [ ]:
hdpmodel,hdptopics=fpw.hdp(corpus2,dictionary)
punteggioHDP=fpw.punteggioHDP(hdptopics,parole)
risultatoTopicHDP=fpw.result_TopicHDP(hdptopics,parole)